In [2]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor

# Function to extract MFCC features and their deltas
def extract_mfcc_features(file_path, n_mfcc=20, delta_order=2):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_delta = librosa.feature.delta(mfcc, order=1)
    mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
    features = np.vstack([mfcc, mfcc_delta, mfcc_delta2])
    features = np.mean(features, axis=1)  # Averaging over time frames
    return features

# Function to extract Log Mel Spectrogram
def extract_mel_spectrogram(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    return np.mean(log_mel_spectrogram, axis=1)

# Function to extract Chroma feature
def extract_chroma(file_path):
    y, sr = librosa.load(file_path, sr=None)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, n_chroma=12)
    return np.mean(chroma, axis=1)

# Function to extract Spectral Contrast feature
def extract_spectral_contrast(file_path):
    y, sr = librosa.load(file_path, sr=None)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, n_bands=6)
    return np.mean(spectral_contrast, axis=1)

# Function to extract Zero Crossing Rate
def extract_zero_crossing_rate(file_path):
    y, sr = librosa.load(file_path, sr=None)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
    return np.mean(zero_crossing_rate, axis=1)

# Set file paths based on the Kaggle competition directory structure
train_file = '/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv'
test_file = '/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv'
train_audio_dir = '/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train'
test_audio_dir = '/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test'

# Load the CSV files
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

# Extract features from the audio files in the 'audios_train' folder (training data)
X_train = []
y_train = []

for idx, row in train_df.iterrows():
    file_path = os.path.join(train_audio_dir, row['filename'])
    if os.path.exists(file_path):  # Ensure the file exists
        mfcc_features = extract_mfcc_features(file_path)
        mel_features = extract_mel_spectrogram(file_path)
        chroma_features = extract_chroma(file_path)
        spectral_contrast_features = extract_spectral_contrast(file_path)
        zero_crossing_features = extract_zero_crossing_rate(file_path)

        X_train.append(np.concatenate([mfcc_features, mel_features, chroma_features, spectral_contrast_features, zero_crossing_features]))
        y_train.append(row['label'])

X_train = np.array(X_train)
y_train = np.array(y_train)

# Scale the features to improve model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Hyperparameter tuning for Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(random_state=42)
gb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
}
gb_grid_search = GridSearchCV(gb_model, gb_param_grid, cv=3, scoring='neg_mean_absolute_error')
gb_grid_search.fit(X_train_scaled, y_train)

# Train XGBoost Model
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}
xgb_grid_search = RandomizedSearchCV(xgb_model, xgb_param_grid, n_iter=10, cv=3, scoring='neg_mean_absolute_error', random_state=42)
xgb_grid_search.fit(X_train_scaled, y_train)

# Train LightGBM Model
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
}
lgb_grid_search = RandomizedSearchCV(lgb.LGBMRegressor(random_state=42), lgb_param_grid, n_iter=10, cv=3, scoring='neg_mean_absolute_error', random_state=42)
lgb_grid_search.fit(X_train_scaled, y_train)

# Train CatBoost Model
cat_model = CatBoostRegressor(random_state=42, silent=True)
cat_model.fit(X_train_scaled, y_train)

# Extract features from the audio files in the 'audios_test' folder (test data)
X_test = []

for filename in test_df['filename']:
    file_path = os.path.join(test_audio_dir, filename)
    if os.path.exists(file_path):  # Ensure the file exists
        mfcc_features = extract_mfcc_features(file_path)
        mel_features = extract_mel_spectrogram(file_path)
        chroma_features = extract_chroma(file_path)
        spectral_contrast_features = extract_spectral_contrast(file_path)
        zero_crossing_features = extract_zero_crossing_rate(file_path)

        X_test.append(np.concatenate([mfcc_features, mel_features, chroma_features, spectral_contrast_features, zero_crossing_features]))

X_test = np.array(X_test)

# Scale the test features using the same scaler as the training set
X_test_scaled = scaler.transform(X_test)

# Make Predictions
y_pred_gb = gb_grid_search.best_estimator_.predict(X_test_scaled)
y_pred_xgb = xgb_grid_search.best_estimator_.predict(X_test_scaled)
y_pred_lgb = lgb_grid_search.best_estimator_.predict(X_test_scaled)
y_pred_cat = cat_model.predict(X_test_scaled)

# Stack the predictions for test set predictions
y_pred_stacked = np.column_stack((y_pred_gb, y_pred_xgb, y_pred_lgb, y_pred_cat))

# Train a meta-model on the predictions (Stacking)
stacking_model = LinearRegression()
stacking_model.fit(y_pred_stacked, y_train[:len(y_pred_stacked)])  # Only fit on the same number of samples

# Final Predictions using the stacking model
y_pred_final = stacking_model.predict(y_pred_stacked)

# Prepare Submission File
submission_final = pd.DataFrame({'filename': test_df['filename'], 'label': y_pred_final})
submission_final.to_csv('/kaggle/working/submission_final.csv', index=False)

print("Final Ensemble Submission file has been generated.")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000964 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20548
[LightGBM] [Info] Number of data points in the train set: 296, number of used features: 208
[LightGBM] [Info] Start training from score 4.239865
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,